# 線形計画法/量子アニーリングマシンでスーパコンピュータ設計
## 背景
スーパーコンピュータおよびHPC(High Performance Computing)クラスタ（以下、HPCシステム）は、CPU、メモリ、アクセラレータ、インターコネクトなど様々な部品から構成される。
システム設計を行う際、予算および電力容量内で、要求される性能（理論演算性能、コア数、シングルスレッド性能、ノード数、アクセラレータの有無など計算対象の分野によって優先度が異なる）最大化するシステム設計が必要となる。人手で全ての細かい部品について入れ替えて検討することは、組み合わせ数が膨大であり、仕様策定の限られた期間内では困難である。
そこで、このHPCシステム設計を最適化問題として捉え、ヒューリスティックな解法により求める性能の最適構成を高速に手に入れることを目的とする。

この手法は、入力するデータを変更することでHPCシステムに限らず、様々な構成を提案することができる。
D-Waveマシンなどと連携した高速な構成提案システムを検討する。


# CPUに関して検討してみる
様々な部品の組み合わせが考えられるが、計算機ノード内の性能や消費電力に大きく影響を与える主要な部品であるCPUの組み合わせを検討してみる。

## CPUデータの取得
### Intel
Processor関係の製品情報の比較ページでデータをエクスポートすることができる。このデータをtsv形式で保存し、pandasで読み込んで利用する。

旧版？( ark.intel.com )のサイトだと定価ベースのデータも取得できる。エクスポートできるデータはHTML形式なので -> tsv -> utf-8への変換を行う必要がある。
https://ark.intel.com/content/www/us/en/ark/compare.html?productIds=215286,215272,215281,215279,215271,215285,215284,215282,212461,215274,212458,212286,215273,215280,212285,212633,215278,215276,212457,212456,212460,212288,217215,212307,212454,212284,212282,212308,212459,217216,212455,212289,212287,215275,215277,215283,215269,215270,204091,204097,204089,205688,204086,204090,204088,205687,204095,204092,204094,204096,205683,204087,205684

新版( www.intel.com )のサイトだと、ログインすると現在価格が表示される（販売されていないものはnanになる）そのままCSVをエクスポートできる。
https://www.intel.com/content/www/us/en/products/compare.html?productIds=199344,197100,197098,193390,193553,193394,193389,193385,193384,193382,199349,215286,212285,212286,212456,212457,212458,212460,212461,212633,215271,215272,215273,215274,215276,215278,215279,215280,215281,215282,215284,215285,204091,204088,204086,204090,204095,205687,217216,212282,212284,212287,212288,212289,212307,212454,212455,212459,217215,212308,205688,204089,204097,205683,204087,204092,204094,204096,205684,195437,195434,194146,194145,192465,192467,192470,192472,192474,192475,192478,192480,192481,192482,192476,120497,120499,120500,120501,120503,120504,120506,120508,123543,123687,125056,120496


AMDで取得できるcsvには価格データがないため、**今回は新版のサイトから取得したデータを使用する。**

### データの読み込み

In [1]:
import pandas as pd
# データ読み込み

## intel
df_intel = pd.read_csv('data/Intel_UPE_ComparisonChart_2022_04_15.csv', skiprows=1, header=2, index_col=0)


In [2]:
# intel データの確認
df_intel

,8180,8153,8156,8158,8160,8164,8168,8170,8176,8160T,...,6338T,5318S,6336Y,5318N,6312U,5320T,5320,5315Y,8352M,8362
Processor Number,,,,,,,,,,,,,,,,,,,,,
Status,Launched,Launched,Launched,Launched,Launched,Launched,Launched,Launched,Launched,Launched,...,Launched,Launched,Launched,Launched,Launched,Launched,Launched,Launched,Launched,Launched
Launch Date,Q3'17,Q3'17,Q3'17,Q3'17,Q3'17,Q3'17,Q3'17,Q3'17,Q3'17,Q3'17,...,Q2'21,Q2'21,Q2'21,Q2'21,Q2'21,Q2'21,Q2'21,Q2'21,Q2'21,Q2'21
Lithography,14 nm,14 nm,14 nm,14 nm,14 nm,14 nm,14 nm,14 nm,14 nm,14 nm,...,10 nm,10 nm,10 nm,10 nm,10 nm,10 nm,10 nm,10 nm,10 nm,10 nm
Use Conditions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Communications Commercial Temp,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Total Cores,28,16,4,12,24,26,24,26,28,24,...,24,24,24,24,24,20,26,8,32,32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Intel® Software Guard Extensions (Intel® SGX),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Yes with Intel® SPS,Yes with Intel® SPS,Yes with Intel® SPS,Yes with Intel® SPS,Yes with Intel® SPS,Yes with Intel® SPS,Yes with Intel® SPS,Yes with Intel® SPS,Yes with Intel® SPS,Yes with Intel® SPS
Intel® Total Memory Encryption,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes
Intel® Platform Firmware Resilience Support,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes


In [3]:
# 使用するデータの整形
# 計算用にnumpy arrayに変換する

#性能比較に使いやすいデータの抽出
df_intel_subset = df_intel.loc[['Total Cores', 'Total Threads', 'Max Turbo Frequency', 'Processor Base Frequency', 'Cache', 'TDP', 'Max Memory Size (dependent on memory type)', 'Maximum Memory Speed', 'Scalability', 'Max # of PCI Express Lanes']]

#df_intel_subset


# Total Coresからndarray作成
totalCores = df_intel_subset.loc['Total Cores'].values

#print(type(totalCores))
#print(totalCores)


# Max Turbo Frequencyから単位[GHz]を除外したndarray作成
# 単位なしでGHzとして扱う
tmp = df_intel_subset.loc['Max Turbo Frequency'].str.split(expand=True)
tmp1 = tmp[0].astype('float').mul(tmp[1].map({'MHz': 0.001, 'GHz': 1}))
maxTurboFreq = tmp1.values

#print(type(maxTurboFreq))
#print(maxTurboFreq)


# Processor Base Frequancyから単位[GHz]を除外したndarray作成
# 単位なしでGHzとして扱う
tmp = df_intel_subset.loc['Processor Base Frequency'].str.split(expand=True)
tmp1 = tmp[0].astype('float').mul(tmp[1].map({'MHz': 0.001, 'GHz': 1}))
procBaseFreq = tmp1.values

#print(type(procBaseFreq))
#print(procBaseFreq)


# Cacheから単位[MB]を除外したndarray作成
tmp = df_intel_subset.loc['Cache'].str.split(expand=True)
tmp1 = tmp[0].astype('float').mul(tmp[1].map({'KB': 0.001, 'MB': 1}))
cache = tmp1.values

#print(cache.shape)
#print(cache)


# TDPから単位[W]を除外したndarray作成
tmp = df_intel_subset.loc['TDP'].str.split(expand=True)
tmp1 = tmp[0].astype('float').mul(tmp[1].map({'W': 1}))
tdp = tmp1.values

#print(tdp)


# Max Memory Sizeから単位[GB], [TB]を除外したndarray作成
tmp = df_intel_subset.loc['Max Memory Size (dependent on memory type)'].str.split(expand=True)
tmp1 = tmp[0].astype('float').mul(tmp[1].map({'GB': 1, 'TB': 1024}))
tmp2 = tmp1.fillna(768)     # nanは768 GBとして扱う
maxMemSize = tmp2.values

#print(maxMemSize)


# Max Memory speedから単位[MHz]を除外したndarray作成
tmp = df_intel_subset.loc['Maximum Memory Speed'].str.split(expand=True)
tmp1 = tmp[0].astype('float').mul(tmp[1].map({'MHz': 1, 'GHz': 1024}))
tmp2 = tmp1.fillna(2933)    # nanは2933 MHzとして扱う
maxMemSpeed = tmp2.values

#print(maxMemSpeed)


# Scalabilityからソケット数上限を抽出
# S8S: Scalable 8 Sockets, 4S: 4 Sockets, ... nan: 1S Onlyに置換
tmp = df_intel_subset.loc['Scalability']
tmp1 = tmp.fillna('1S Only')    # nanは1S Onlyとして扱う
scalability = tmp1.map({'S8S': 8, '4S': 4, '2S': 2, '1S Only': 1}).values

#print(scalability)


# PCI Express Lanesからndarray作成
pcieLanes = df_intel_subset.loc['Max # of PCI Express Lanes'].values

#print(type(pcieLanes))
#print(pcieLanes)



### AMD
プロセッサー仕様のWebページから、csvデータを取得できる。このデータをpandasで読み込んで利用する。

https://www.amd.com/ja/products/specifications/processors

In [4]:
#import pandas as pd
# データ読み込み

## AMD
#df_amd = pd.read_csv('data/AMD_EPYC_Comparison_2022_04_13.csv', header=0, index_col=0)

# AMD データの確認
#df_amd

# 線形計画法で解くアイディア

## 線形計画法

### 問題設定
まずは1種類のCPUで1つのシステムの性能を最大化することを目指す。
この時、電力の制約を満たすこと、ノードあたりのCPUコア数の制約条件を満足すること。

### 定数定義:
- $ P_i  $ :CPU iの演算性能の指標として、(性能P) = (core数) $\times$ (周波数)を使う。
- $ C_i $ :CPU iのCashe容量
- $ E_i $ :CPU iの消費電力
- $ E_{max} $ :システム最大消費電力
- $ T_i $ :CPU iのコア数/Thread数
- $ T_{sys} $ :CPUあたりのコア数 ノードのコア数をn以上やn以下を指定する場合に使用する
- $ S_i $ :CPUの対応ソケット数
- $ S_{sys} $ :システムの対応ソケット数 ノードあたりいくつのCPUを想定しているか

### 決定変数:
- $ x_i $ その種類の数量(整数) $x_i = 0, 1, 2, ...$
- $ z_i $ CPUの種類、採用/採用しない {0 , 1} システム内のCPUを1種類に制限するため使う
- $ y_i = x_i z_i $ 積による表現を目的関数に用いる。

($i = 1,2, ... , N$ CPUの種類)


### 目的関数 (objective):
- 性能の最大化

$$ \max \sum_i^N P_i y_i $$

- キャッシュ容量最大化

$$ \max \sum_i^N C_i y_i $$


### 制約条件 (subject to):
- 電力は設備の最大消費電力以下

$$ \sum_i^N E_i y_i \leq E_{max} $$

- CPUあたりのコア数 <= コア数制限

$$ \sum_i^N T_i z_i \leq T_{sys} $$

- CPUあたりのスケーラビリティ >= ノードあたりのソケット数

$$ \sum_i^N S_i z_i \geq S_{sys} $$

- システムは1種類のCPUで構成する

$$ \sum_i^N z_i = 1 $$




In [5]:
import numpy as np
import pulp

num_systems = 1



